In [1]:
import torch
import pandas as pd
import numpy as np
import torch

In [2]:
#with open('data/fragments2.tsv', 'r') as file:
#    for i in range(1000):
#        print(file.readline())

In [3]:
#df = pd.read_csv('data/fragments1.tsv', sep='\t', skiprows=51)
#df = pd.read_csv('data/fragments2.tsv', sep='\t', skiprows=51, header=None)

#df.columns = ['Chromosome', 'Start', 'End', 'Barcode', 'Count']

In [4]:
#df
#df_subset = df.head(100000)

In [5]:
# ADD EMBEDDINGS CREATION        

In [6]:
#embeddingsTot = embed()  ## dont run, first add embeddings creation

#embeddings = embeddingsTot[0]
#cnv = embeddingsTot[1]
#open_cromatin = embeddingsTot[2]

In [7]:
seq_len = 10000
num_samples = 100

embeddings_train = np.random.rand(num_samples, seq_len, 4)  # 100 samples, each of length 6000 and 4 features
cnv_train = np.random.rand(num_samples, seq_len, 2)  # 100 samples, each of length 6000 and 4 features
open_cromatin_train = np.random.rand(num_samples, seq_len, 1)  # 100 samples, each of length 6000 and 4 features
gene_expression_train = np.random.choice(["high", "low"], size=(num_samples, 1))

embeddings_train_tensor = torch.tensor(embeddings_train, dtype=torch.float32)
cnv_train_tensor = torch.tensor(cnv_train, dtype=torch.float32)
open_cromatin_train_tensor = torch.tensor(open_cromatin_train, dtype=torch.float32)

stacked_data_train = torch.cat((embeddings_train_tensor, cnv_train_tensor, open_cromatin_train_tensor), dim=-1)

embeddings_val = np.random.rand(num_samples, seq_len, 4)  # 100 samples, each of length 6000 and 4 features
cnv_val = np.random.rand(num_samples, seq_len, 2)  # 100 samples, each of length 6000 and 4 features
open_cromatin_val = np.random.rand(num_samples, seq_len, 1)  # 100 samples, each of length 6000 and 4 features
gene_expression_val = np.random.choice(["high", "low"], size=(num_samples, 1))

embeddings_val_tensor = torch.tensor(embeddings_val, dtype=torch.float32)
cnv_val_tensor = torch.tensor(cnv_val, dtype=torch.float32)
open_cromatin_val_tensor = torch.tensor(open_cromatin_val, dtype=torch.float32)

stacked_data_val = torch.cat((embeddings_val_tensor, cnv_val_tensor, open_cromatin_val_tensor), dim=-1)

embeddings_test = np.random.rand(num_samples, seq_len, 4)  # 100 samples, each of length 6000 and 4 features
cnv_test = np.random.rand(num_samples, seq_len, 2)  # 100 samples, each of length 6000 and 4 features
open_cromatin_test = np.random.rand(num_samples, seq_len, 1)  # 100 samples, each of length 6000 and 4 features
gene_expression_test = np.random.choice(["high", "low"], size=(num_samples, 1))

embeddings_test_tensor = torch.tensor(embeddings_test, dtype=torch.float32)
cnv_test_tensor = torch.tensor(cnv_test, dtype=torch.float32)
open_cromatin_test_tensor = torch.tensor(open_cromatin_test, dtype=torch.float32)

stacked_data_test = torch.cat((embeddings_test_tensor, cnv_test_tensor, open_cromatin_test_tensor), dim=-1)

In [8]:
gene_expression_train_b = torch.tensor([1 if label == "high" else 0 for label in gene_expression_train]).float().view(-1, 1)
gene_expression_val_b = torch.tensor([1 if label == "high" else 0 for label in gene_expression_val]).float().view(-1, 1)
gene_expression_test_b = torch.tensor([1 if label == "high" else 0 for label in gene_expression_test]).float().view(-1, 1)

In [9]:
from torch.utils.data import TensorDataset, DataLoader, random_split, Dataset
from torch.cuda.amp import GradScaler, autocast
import torch.nn as nn
import torch.optim as optim
import copy


class StackedDataset(Dataset):
    def __init__(self, ablated_inputs, gene_expression):
        self.ablated_inputs = torch.tensor(ablated_inputs, dtype=torch.float32)
        self.gene_expression = torch.tensor(gene_expression, dtype=torch.float32)

    def __len__(self):
        return len(self.ablated_inputs)

    def __getitem__(self, idx):
        ablated_inputs = self.ablated_inputs[idx]
        gene_expression = self.gene_expression[idx]
        
        return ablated_inputs, gene_expression
    
    
train_dataset = StackedDataset(stacked_data_train, gene_expression_train_b)
val_dataset = StackedDataset(stacked_data_val, gene_expression_val_b)
test_dataset = StackedDataset(stacked_data_test, gene_expression_test_b)

BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

C:\Users\Aliprandi\anaconda3\envs\tp\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Aliprandi\anaconda3\envs\tp\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


In [10]:
import torch.nn.functional as F

class ChromosomeCNN(nn.Module):
    def __init__(self,  input_dim, seq_len, output_dim):
        super(ChromosomeCNN, self).__init__()
        
        self.input_dim = input_dim
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=2)

        self.fc1 = None 
        self.fc2 = nn.Linear(128, 1)
        
        self.seq_len = seq_len
        
    def initialize_fc1(self, x):

        if self.fc1 is None: 
            flattened_size = x.shape[1] * x.shape[2] 
            self.fc1 = nn.Linear(flattened_size, 128).to(x.device)     
    
    def forward(self, inputs_seq):
        
        #print(f"Shape of inputs before permute: {inputs_seq.shape}")
        
        x = inputs_seq.permute(0, 2, 1)
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, start_dim=1)
        
        if self.fc1 is None:

            fc1_input_size = x.shape[1]
            self.fc1 = nn.Linear(fc1_input_size, 128)
                
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x

In [11]:
def create_ablated_dataloader(loader, channel_to_remove, channel_variable_counts):
    ablated_dataloader = []
    
    for batch in loader:
        ablated_inputs, targets = batch
        
        ablated_inputs = ablation_study(ablated_inputs, channel_to_remove, channel_variable_counts)
        
        ablated_dataloader.append((ablated_inputs, targets))
    
        
    return ablated_dataloader

In [12]:
def full_study(loader):
    full_dataloader = []
    
    for batch in loader:
        ablated_inputs, targets = batch
        
        full_dataloader.append((ablated_inputs, targets))
        
    return full_dataloader

In [13]:
def ablation_study(inputs, channel_to_remove, channel_variable_counts):
    # inputs is a tensor containing all the stacked channels
    # channel_variable_counts specifies how many variables each channel contains (e.g., embeddings, cnv, chromatin)

    # Calculate the cumulative index of where each channel starts and ends in the stacked tensor
    start_idx = 0
    ablated_inputs = []
    
    for i, count in enumerate(channel_variable_counts):
        end_idx = start_idx + count
        
        # Only keep the channel if it is not the one being ablated
        if i != channel_to_remove:  
            ablated_inputs.append(inputs[:, :, start_idx:end_idx])  # Slice to keep the relevant channel (batch_size, seq_len, count)
        
        start_idx = end_idx  # Move the start index to the next channel

    # Stack the remaining channels together
    stacked_inputs = torch.cat(ablated_inputs, dim=-1)  # Concatenate along the last dimension (features)

    return stacked_inputs



In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sequ_len = 10000 ##### add correct one

epochs = 2

def train_(model, train_loader, val_loader, epochs):
    
    best_val_loss = float('inf')
    train_losses_avg = []
    val_losses_avg = []
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    

    for epoch in range(epochs):

        model.train()
        
        total_loss = 0
        
        for stacked_inputs_batch, y_batch in train_loader:

            stacked_inputs_batch = stacked_inputs_batch.to(device)
            y_batch = y_batch.to(device, non_blocking=True)
            #stacked_inputs_batch = stacked_inputs_batch.unsqueeze(0)

            optimizer.zero_grad()

            with autocast():   
                outputs = model(stacked_inputs_batch)
                loss = criterion(outputs, y_batch)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
    
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")
    
        model.eval()
        val_losses = []
        for stacked_inputs_batch, y_batch in val_loader:

            stacked_inputs_batch = stacked_inputs_batch.to(device)
            y_batch = y_batch.to(device, non_blocking=True)
            #stacked_inputs_batch = stacked_inputs_batch.unsqueeze(0)

            with torch.no_grad(), autocast():
                y_pred = model(stacked_inputs_batch)
                lossV = criterion(y_pred, y_batch)
                val_losses.append(lossV.item())

        avg_val_loss = sum(val_losses) / len(val_losses)
        val_losses_avg.append(avg_val_loss)
        print(f'Epoch {epoch+1}, Val loss: {avg_val_loss}')
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model = copy.deepcopy(model.state_dict())
            
    return avg_val_loss

In [15]:
def ablation_study_evaluation(train_loader, val_loader, test_loader, channel_variable_counts, seq_len, num_epochs):

    print("Training with all channels intact...")
    num_channels = 7
    
    full_train_loader = full_study(train_loader)
    full_val_loader = full_study(val_loader)
    full_test_loader = full_study(test_loader)
    
    model = ChromosomeCNN(input_dim = num_channels, seq_len = seq_len, output_dim = 1).to(device)
    baseline_loss = train_(model, full_train_loader, full_val_loader, num_epochs)
    
    torch.save({
        'model_state_dict': model.state_dict(),
    }, 'baseline_model.pth')
    
    baseline_test = test_model("baseline_model.pth", full_test_loader, num_channels, seq_len)

    for channel_idx in range(3):
        print(f"\nAblating channel {channel_idx}...")
        
        remaining_channels = [i for i in range(3) if i != channel_idx]
        remaining_variables = sum(channel_variable_counts[i] for i in remaining_channels)
        print("remaing variables", remaining_variables)
        
        model = ChromosomeCNN(input_dim = remaining_variables, seq_len = seq_len, output_dim = 1).to(device)
        
        ablated_train_loader = create_ablated_dataloader(train_loader, channel_idx, channel_variable_counts)
        ablated_val_loader = create_ablated_dataloader(val_loader, channel_idx, channel_variable_counts)
        ablated_test_loader = create_ablated_dataloader(test_loader, channel_idx, channel_variable_counts)

        
        model_ablated = ChromosomeCNN(input_dim=remaining_variables, seq_len=seq_len, output_dim=1).to(device)
        ablated_model_name = f"ablated_model_channel_{channel_idx}"
        
        ablated_loss = train_(model_ablated, ablated_train_loader, ablated_val_loader, epochs)#, ablated_model_name)
        
        ablated_model_filename = f'ablated_model_channel_{channel_idx}.pth'
        torch.save({
            'model_state_dict': model_ablated.state_dict(),
        }, ablated_model_filename)

        results = {}
        results[f"Ablated Channel {channel_idx}"] = test_model(
            f"{ablated_model_name}.pth", ablated_test_loader, remaining_variables, seq_len
        )
        
        
        print(f"Loss after ablating channel {channel_idx}: {ablated_loss:.4f}")
        print(f"Performance drop: {baseline_loss - ablated_loss:.4f}")

In [16]:
def test_model(model_path, test_loader, total_variables, seq_len):

    model = ChromosomeCNN(input_dim=total_variables, seq_len=seq_len, output_dim=1).to(device)
    checkpoint = torch.load(model_path)
    
    input_tensor = torch.zeros(1, model.seq_len, model.input_dim).to(device)
    model(input_tensor)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    model.eval()

    criterion = nn.BCEWithLogitsLoss()
    test_losses = []

    with torch.no_grad():
        for stacked_inputs_batch, y_batch in test_loader:
            stacked_inputs_batch = stacked_inputs_batch.to(device)
            y_batch = y_batch.to(device, non_blocking=True)
            #stacked_inputs_batch = stacked_inputs_batch.unsqueeze(0)

            with autocast():
                outputs = model(stacked_inputs_batch)
                loss = criterion(outputs, y_batch)
                test_losses.append(loss.item())

    avg_test_loss = sum(test_losses) / len(test_losses)
    print(f"Test MSE: {avg_test_loss:.4f}")
    return avg_test_loss

In [17]:
embedding_dim = 4  
cnv_dim = 2        
chromatin_dim = 1  
expression_dim = 1

ablation_study_evaluation(train_loader, val_loader, test_loader, channel_variable_counts=[embedding_dim, cnv_dim, chromatin_dim, expression_dim], seq_len=seq_len, num_epochs=epochs)


Training with all channels intact...


C:\Users\Aliprandi\anaconda3\envs\tp\lib\site-packages\torch\amp\autocast_mode.py:202: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Epoch 1/2, Loss: 0.6922
Epoch 1, Val loss: 0.6925352811813354
Epoch 2/2, Loss: 0.6915
Epoch 2, Val loss: 0.6924791187047958
Test MSE: 0.6941

Ablating channel 0...
remaing variables 3
Epoch 1/2, Loss: 0.6966
Epoch 1, Val loss: 0.6913031488656998
Epoch 2/2, Loss: 0.6960
Epoch 2, Val loss: 0.6913331151008606
Test MSE: 0.6905
Loss after ablating channel 0: 0.6913
Performance drop: 0.0011

Ablating channel 1...
remaing variables 5
Epoch 1/2, Loss: 0.6924
Epoch 1, Val loss: 0.6914888471364975
Epoch 2/2, Loss: 0.6918
Epoch 2, Val loss: 0.6914166957139969
Test MSE: 0.6922
Loss after ablating channel 1: 0.6914
Performance drop: 0.0011

Ablating channel 2...
remaing variables 6
Epoch 1/2, Loss: 0.6922
Epoch 1, Val loss: 0.6943075805902481
Epoch 2/2, Loss: 0.6915
Epoch 2, Val loss: 0.6943929493427277
Test MSE: 0.6922
Loss after ablating channel 2: 0.6944
Performance drop: -0.0019


In [18]:
#model.eval()
#correct = 0
#total = 0
#with torch.no_grad():
#    for X_batch, y_batch in test_loader:
#        X_batch, y_batch = X_batch.to(device).unsqueeze(1), y_batch.to(device).unsqueeze(1)
#        outputs = model(X_batch)
#        predictions = (outputs > 0.5).float()
#        correct += (predictions == y_batch).sum().item()
#        total += y_batch.size(0)

#accuracy = correct / total
#print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [19]:
#model.load_state_dict(best_model)
#model.eval()
#test_losses = []
#y_preds = []
#y_actuals = []

#scaler = GradScaler()

#for X_batch, cnv_batch, y_batch in test_loader:

#    X_batch = X_batch.unsqueeze(1).to(device, non_blocking=True)
#    cnv_batch = cnv_batch.to(device)
#    y_batch = y_batch.to(device, non_blocking=True)
    
#    with torch.no_grad(), autocast():
#        y_pred = model(X_batch, cnv_batch)
#        lossV = criterion(y_pred, y_batch)
        
#        y_preds.extend(y_pred.cpu().numpy())
#        y_actuals.extend(y_batch.cpu().numpy())
#        test_losses.append(lossV.item())

#avg_test_loss = sum(test_losses) / len(test_losses)
#print(f'Test MSE: {avg_test_loss}')

In [20]:
def model_summary(model):
    print("Model Summary:")
    print("{:<50} {:<30} {:<15} {:<15}".format("Layer Name", "Shape", "Parameters", "Trainable"))
    print("-" * 110)
    total_params = 0
    total_trainable_params = 0
    lm_params = 0
    lm_trainable_params = 0
    lm_layers = 0
    for name, parameter in model.named_parameters():
        param = parameter.numel()
        total_params += param
        # Check if the parameter is trainable
        trainable = parameter.requires_grad
        trainable_param = param if trainable else 0
        total_trainable_params += trainable_param
        print("{:<50} {:<30} {:<15} {:<15}".format(name, str(parameter.size()), param, trainable_param))
    print("-" * 110)
    print(f"Total Parameters: {total_params}")
    print(f"Trainable Parameters: {total_trainable_params}")

#model_summary(model)